#STEP 1 — Open Dataset Safely

In [2]:
import pandas as pd

# Load only 100k rows (safe for laptop)
df = pd.read_csv("flights.csv", nrows=100000)

# View first rows
print(df.head())


      FL_DATE                AIRLINE                AIRLINE_DOT AIRLINE_CODE  \
0  2019-01-09  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
1  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
2  2022-07-22  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
3  2023-03-06   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
4  2020-02-23       Spirit Air Lines       Spirit Air Lines: NK           NK   

   DOT_CODE  FL_NUMBER ORIGIN          ORIGIN_CITY DEST  \
0   19977.0     1562.0    FLL  Fort Lauderdale, FL  EWR   
1   19790.0     1149.0    MSP      Minneapolis, MN  SEA   
2   19977.0      459.0    DEN           Denver, CO  MSP   
3   19790.0     2295.0    MSP      Minneapolis, MN  SFO   
4   20416.0      407.0    MCO          Orlando, FL  DFW   

               DEST_CITY  ...  DIVERTED  CRS_ELAPSED_TIME  ELAPSED_TIME  \
0             Newark, NJ  ...       0.0             186.0         176.0   
1            S

#STEP 2 — Understand Dataset

In [3]:
# Structure of data
print(df.info())

# Basic stats
print(df.describe())

# Check missing values
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15361 entries, 0 to 15360
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   FL_DATE                  15361 non-null  object 
 1   AIRLINE                  15361 non-null  object 
 2   AIRLINE_DOT              15361 non-null  object 
 3   AIRLINE_CODE             15361 non-null  object 
 4   DOT_CODE                 15360 non-null  float64
 5   FL_NUMBER                15360 non-null  float64
 6   ORIGIN                   15360 non-null  object 
 7   ORIGIN_CITY              15360 non-null  object 
 8   DEST                     15360 non-null  object 
 9   DEST_CITY                15360 non-null  object 
 10  CRS_DEP_TIME             15360 non-null  float64
 11  DEP_TIME                 14990 non-null  float64
 12  DEP_DELAY                14990 non-null  float64
 13  TAXI_OUT                 14986 non-null  float64
 14  WHEELS_OFF            

📌 Data Quality Observations
#1️⃣ Date Column

FL_DATE is stored as an object (string) datatype.
This column needs to be converted into datetime format for proper time-based analysis and feature extraction.

➡ Action Taken: Converted to datetime format.

#2️⃣ Delay Columns with Missing Values

DEP_DELAY contains 371 missing values.
ARR_DELAY contains 405 missing values.

Missing values in delay columns can affect delay analysis and statistics.

➡ Action Taken: Filled missing values using the median of each column.

#3️⃣ Cancellation Code Mostly Empty

CANCELLATION_CODE has 14,986 missing values.
This indicates that most flights were not cancelled.

➡ Action Taken: Filled missing values with "Not Cancelled".

#4️⃣ Delay Reason Columns Mostly Empty

Delay reason columns such as:

DELAY_DUE_CARRIER

DELAY_DUE_WEATHER

DELAY_DUE_NAS

DELAY_DUE_SECURITY

DELAY_DUE_LATE_AIRCRAFT

have many missing values.

These missing values usually indicate that no delay occurred for that reason.

➡ Action Taken: Filled missing values with 0.

#STEP 3 — Fix Date Format

In [4]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])


In [5]:
print(df['FL_DATE'].head())


0   2019-01-09
1   2022-11-19
2   2022-07-22
3   2023-03-06
4   2020-02-23
Name: FL_DATE, dtype: datetime64[ns]


#STEP 4 — Handle Missing Values
**Fill delay columns with median**

In [7]:
df['DEP_DELAY'] = df['DEP_DELAY'].fillna(df['DEP_DELAY'].median())
df['ARR_DELAY'] = df['ARR_DELAY'].fillna(df['ARR_DELAY'].median())


**Fill cancellation code**

In [9]:
df['CANCELLATION_CODE'] = df['CANCELLATION_CODE'].fillna("Not Cancelled")


**Fill delay reasons with 0**

In [10]:
delay_cols = [
    'DELAY_DUE_CARRIER',
    'DELAY_DUE_WEATHER',
    'DELAY_DUE_NAS',
    'DELAY_DUE_SECURITY',
    'DELAY_DUE_LATE_AIRCRAFT'
]

df[delay_cols] = df[delay_cols].fillna(0)


#STEP 5 — Feature Engineering

**Month**

In [11]:
df['MONTH'] = df['FL_DATE'].dt.month


**Day of Week**

In [12]:
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek


**Hour (from DEP_TIME)**-The
**time is in HHMM format.**

In [13]:
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'], format='%H%M', errors='coerce')

df['HOUR'] = df['DEP_TIME'].dt.hour


**Route Feature**

In [14]:
df['ROUTE'] = df['ORIGIN'] + "-" + df['DEST']


#STEP 6 — Remove Duplicates

In [15]:
df.drop_duplicates(inplace=True)


#STEP 7 — Ensure Positive Delays

In [16]:
df = df[df['ARR_DELAY'] >= 0]
df = df[df['DEP_DELAY'] >= 0]


#STEP 8 — Normalize Airline Names

In [17]:
df['AIRLINE'] = df['AIRLINE'].str.upper().str.strip()


#STEP 9 — Save Cleaned File

In [19]:
print(df.isnull().sum())


FL_DATE                    0
AIRLINE                    0
AIRLINE_DOT                0
AIRLINE_CODE               0
DOT_CODE                   0
FL_NUMBER                  0
ORIGIN                     0
ORIGIN_CITY                0
DEST                       0
DEST_CITY                  0
CRS_DEP_TIME               0
DEP_TIME                   8
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON                  0
TAXI_IN                    0
CRS_ARR_TIME               0
ARR_TIME                   0
ARR_DELAY                  0
CANCELLED                  0
CANCELLATION_CODE          0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE                   0
DELAY_DUE_CARRIER          0
DELAY_DUE_WEATHER          0
DELAY_DUE_NAS              0
DELAY_DUE_SECURITY         0
DELAY_DUE_LATE_AIRCRAFT    0
MONTH                      0
DAY_OF_WEEK                0
HOUR          

In [20]:
df.dropna(subset=['DEP_TIME','HOUR'], inplace=True)


In [21]:
print(df.isnull().sum())


FL_DATE                    0
AIRLINE                    0
AIRLINE_DOT                0
AIRLINE_CODE               0
DOT_CODE                   0
FL_NUMBER                  0
ORIGIN                     0
ORIGIN_CITY                0
DEST                       0
DEST_CITY                  0
CRS_DEP_TIME               0
DEP_TIME                   0
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON                  0
TAXI_IN                    0
CRS_ARR_TIME               0
ARR_TIME                   0
ARR_DELAY                  0
CANCELLED                  0
CANCELLATION_CODE          0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE                   0
DELAY_DUE_CARRIER          0
DELAY_DUE_WEATHER          0
DELAY_DUE_NAS              0
DELAY_DUE_SECURITY         0
DELAY_DUE_LATE_AIRCRAFT    0
MONTH                      0
DAY_OF_WEEK                0
HOUR          

In [22]:
df.to_csv("cleaned_flights.csv", index=False)
